In [1]:
from ctgan import CTGAN
from ctgan import load_demo
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import torch
import tensorflow as tf
print(torch.cuda.is_available())
import pandas as pd
import numpy as np

True


In [2]:
real_data = pd.read_csv("ciciot2023.csv")

In [3]:
scaler = StandardScaler()
label = LabelEncoder()
# data[data.columns[:-1]] = scaler.fit_transform(data[data.columns[:-1]])
#data[data.columns[:-1]] = np.array(data[data.columns[:-1]], dtype=float)
real_data.replace([np.inf, -np.inf], np.nan, inplace=True)
#data.iloc[:, -1] = label.fit_transform(data.iloc[:, -1])
real_data.dropna(inplace=True)
real_data.head(3).T

,0,1,2
flow_duration,1.544575,0.106292,7.539975
Header_Length,2247048.0,119847.6,520790.5
Protocol Type,6.0,6.0,6.0
Duration,64.0,75.8,64.0
Rate,967.208942,707.484659,65.848973
Srate,967.208942,707.484659,65.848973
Drate,0.0,0.0,0.0
fin_flag_number,0.0,0.0,0.0
syn_flag_number,0.0,0.0,0.0
rst_flag_number,0.0,0.0,0.0


In [4]:
def get_discrete_columns(dataframe, max_unique_values=10, convert_to_string=False):
    """
    Hàm để xác định các cột rời rạc (discrete), bao gồm các cột có số lượng giá trị unique ít hơn hoặc bằng max_unique_values.
    Ngoài ra, có thể chuyển đổi kiểu dữ liệu của các cột rời rạc sang string.
    
    - dataframe: DataFrame chứa dữ liệu
    - max_unique_values: Số lượng giá trị unique tối đa để coi là discrete (mặc định là 80)
    - convert_to_string: Nếu True, chuyển kiểu dữ liệu của các cột rời rạc sang string.
    
    Trả về:
    - discrete_columns: Danh sách tên các cột rời rạc.
    - discrete_indices: Danh sách chỉ số (index) của các cột rời rạc.
    """
    discrete_columns = []
    discrete_indices = []
    
    for col in dataframe.columns:
        unique_values = dataframe[col].nunique()
        print(f"Column '{col}' - Number of unique values: {unique_values}")
        # Chọn các cột có kiểu dữ liệu số và số lượng giá trị unique nhỏ hơn hoặc bằng max_unique_values
        if dataframe[col].dtype in ['float64', 'int64'] and unique_values <= max_unique_values:
            discrete_columns.append(col)
            discrete_indices.append(dataframe.columns.get_loc(col))
            
            # Nếu yêu cầu, chuyển đổi kiểu dữ liệu sang string
            if convert_to_string:
                dataframe[col] = dataframe[col].astype(str)
                print(f"Column '{col}' converted to string.")
    
    return discrete_columns, discrete_indices

# Ví dụ với dataset của bạn
discrete_columns, discrete_indices = get_discrete_columns(real_data)
discrete_columns.append("Target")
print("Discrete Columns:", discrete_columns)
print("Indices of Discrete Columns:", discrete_indices)

Column 'flow_duration' - Number of unique values: 419327
Column 'Header_Length' - Number of unique values: 152840
Column 'Protocol Type' - Number of unique values: 2508
Column 'Duration' - Number of unique values: 5592
Column 'Rate' - Number of unique values: 794454
Column 'Srate' - Number of unique values: 794454
Column 'Drate' - Number of unique values: 56
Column 'fin_flag_number' - Number of unique values: 2
Column 'syn_flag_number' - Number of unique values: 2
Column 'rst_flag_number' - Number of unique values: 2
Column 'psh_flag_number' - Number of unique values: 2
Column 'ack_flag_number' - Number of unique values: 2
Column 'ece_flag_number' - Number of unique values: 1
Column 'cwr_flag_number' - Number of unique values: 1
Column 'ack_count' - Number of unique values: 218
Column 'syn_count' - Number of unique values: 629
Column 'fin_count' - Number of unique values: 228
Column 'urg_count' - Number of unique values: 5728
Column 'rst_count' - Number of unique values: 16705
Column '

In [5]:
#real_data = pd.read_csv("iotid20.csv")

# Names of the columns that are discrete
#discrete_columns = [
    #'workclass',
    #'education',
    #'marital-status',
    #'occupation',
    #'relationship',
    #'race',
    ##'native-country',
    #'income'
#]

In [6]:
ctgan = CTGAN()

In [7]:
#ctgan.fit(real_data, discrete_columns)
ctgan.load_model("2023\ctgan_model_epoch_10.pth")

c:\Final Project in Uni\CTGAN\ctgan\synthesizers\ctgan.py:647: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(model_path, map_location=device)


Model loaded from 2023\ctgan_model_epoch_10.pth


In [8]:
import pandas as pd
import numpy as np

def sample_data_by_class(ctgan, csv_path, label_column):
    """
    Sample synthetic data matching the class distribution in the provided dataset.

    Args:
        ctgan: Trained CTGAN model.
        csv_path (str): Path to the dataset (CSV file).
        label_column (str): Name of the column containing the labels.

    Returns:
        pandas.DataFrame: Synthetic data matching the class distribution.
    """
    # Đọc tập dữ liệu gốc
    df = pd.read_csv(csv_path)

    # Đếm số lượng mẫu của từng lớp
    label_counts = df[label_column].value_counts().to_dict()

    synthetic_data = []

    for label, count in label_counts.items():
        print(f"Sampling {count} rows for label '{label}'...")
        # Sử dụng CTGAN để sinh dữ liệu cho nhãn cụ thể
        sampled_data = ctgan.sample_by_label(n=int(count*0.8), label_column=label_column, label_value=label)
        synthetic_data.append(pd.DataFrame(sampled_data))  # Chuyển sang DataFrame

    # Kết hợp dữ liệu đã sinh
    synthetic_data = pd.concat(synthetic_data, ignore_index=True)
    return synthetic_data

synthetic_data = sample_data_by_class(ctgan, "ciciot2023.csv", "label")
#synthetic_data = ctgan.sample(len(real_data))


Sampling 154230 rows for label 'DDoS-ICMP_Flood'...
Sampling 115928 rows for label 'DDoS-UDP_Flood'...
Sampling 96337 rows for label 'DDoS-TCP_Flood'...
Sampling 87707 rows for label 'DDoS-PSHACK_Flood'...
Sampling 86945 rows for label 'DDoS-SYN_Flood'...
Sampling 86647 rows for label 'DDoS-RSTFINFlood'...
Sampling 77070 rows for label 'DDoS-SynonymousIP_Flood'...
Sampling 71082 rows for label 'DoS-UDP_Flood'...
Sampling 57220 rows for label 'DoS-TCP_Flood'...
Sampling 43456 rows for label 'DoS-SYN_Flood'...
Sampling 23522 rows for label 'BenignTraffic'...
Sampling 21245 rows for label 'Mirai-greeth_flood'...
Sampling 19075 rows for label 'Mirai-udpplain'...
Sampling 16100 rows for label 'Mirai-greip_flood'...


In [9]:
# Create synthetic data
#synthetic_data = ctgan.sample(250000)

In [11]:
def sort_by_class(df, class_column):
    """
    Sắp xếp dữ liệu theo từng lớp (class).
    
    Args:
    df (pandas.DataFrame): Dữ liệu cần sắp xếp.
    class_column (str): Tên cột đại diện cho lớp (class).
    
    Returns:
    pandas.DataFrame: Dữ liệu đã được sắp xếp.
    """
    # Sắp xếp dữ liệu theo từng lớp
    return df.groupby(class_column, group_keys=False).apply(lambda x: x.sort_values(by=class_column))  # 'z' là cột bạn muốn sắp xếp trong mỗi lớp
synthetic_data = sort_by_class(synthetic_data,"label")

In [12]:
synthetic_data.to_csv("ctgan_10_perclass_2023.csv", index=False)